In [2]:
# -----------------------库导入--------------------------
import readygo
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------------------图像与显示函数---------------------
image = cv2.imread("picture/0.png")
if image is None:
    raise ValueError("无法读取图片，请确认路径是否正确")

def imshow(img):
    plt.figure(figsize=(5, 5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# ----------------------控件定义------------------------
# 输出窗口
output = widgets.Output()

# LOWEST 使用单值滑块
slider_lowest = widgets.IntSlider(value=100, min=0, max=100, step=1, description='LOWEST:', continuous_update=False)
# 红色 HSV 范围使用区间滑块
slider_red1_h = widgets.IntRangeSlider(value=[0, 10], min=0, max=180, step=1, description='Red1 H:', continuous_update=False)
slider_red1_s = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red1 S:', continuous_update=False)
slider_red1_v = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red1 V:', continuous_update=False)

slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, step=1, description='Red2 H:', continuous_update=False)
slider_red2_s = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red2 S:', continuous_update=False)
slider_red2_v = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red2 V:', continuous_update=False)
# 分组排列
group_red1 = widgets.HBox([slider_red1_h, slider_red2_h])
group_red2 = widgets.HBox([slider_red1_s, slider_red2_s])
group_red3 = widgets.HBox([slider_red1_v, slider_red2_v])

# ----------------------回调函数------------------------
def on_value_change(change=None):
    # 获取 LOWEST 阈值
    LOWEST = slider_lowest.value

    # 获取红色范围
    lower_red1 = np.array([slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_v.value[0]])
    upper_red1 = np.array([slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_v.value[1]])

    lower_red2 = np.array([slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]])
    upper_red2 = np.array([slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_v.value[1]])

    # 调用处理函数
    mask0, mask1, draw = readygo.detect_red_info(image, LOWEST, lower_red1, upper_red1, lower_red2, upper_red2)

    # 显示图像
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB))
        plt.title("Result")
        plt.subplot(1, 2, 2)
        plt.imshow(mask0, cmap='gray')
        plt.title("Mask")
        plt.show()

# 绑定回调
sliders = [slider_lowest, slider_red1_h, slider_red1_s, slider_red1_v,
           slider_red2_h, slider_red2_s, slider_red2_v]
for s in sliders:
    s.observe(on_value_change, names='value')


In [3]:
# ----------------------界面展示------------------------
controls = widgets.VBox([slider_lowest, group_red1, group_red2, group_red3])
display(controls, output)

# ----------------------首次触发------------------------
on_value_change()

Output()

In [1]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

# 视频源
cap = cv2.VideoCapture("video/red-green.mp4")
if not cap.isOpened():
    raise IOError("无法打开视频文件")

# 图像输出部件（原图、处理图、mask）
image_original = widgets.Image(format='jpeg', width=320, height=240)
image_processed = widgets.Image(format='jpeg', width=320, height=240)
image_mask = widgets.Image(format='jpeg', width=320, height=240)

# 控制变量
playing = False
frame_lock = threading.Lock()

# 参数滑块
slider_lowest = widgets.IntSlider(value=100, min=0, max=1000, description='LOWEST:')
slider_red1_h = widgets.IntRangeSlider(value=[0, 15], min=0, max=180, description='Red1 H:')
slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, description='Red2 H:')

# 控制按钮
toggle_button = widgets.ToggleButton(value=False, description="Play/Pause", icon='play')
close_button = widgets.Button(description="关闭", icon='times', button_style='danger')

# 图像处理函数
def process_frame(frame):
    LOWEST = slider_lowest.value
    lower_red1 = np.array([slider_red1_h.value[0], 70, 70])
    upper_red1 = np.array([slider_red1_h.value[1], 255, 255])
    lower_red2 = np.array([slider_red2_h.value[0], 70, 70])
    upper_red2 = np.array([slider_red2_h.value[1], 255, 255])

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    draw = frame.copy()
    for c in contours:
        area = cv2.contourArea(c)
        if LOWEST < area < 1000:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(draw, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return draw, mask

def to_jpeg_bytes(frame, is_mask=False):
    if is_mask:
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    _, jpeg = cv2.imencode('.jpg', frame)
    return jpeg.tobytes()

# 播放线程
def play_video():
    global playing
    while playing:
        with frame_lock:
            ret, frame = cap.read()
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            processed, mask = process_frame(frame)
            image_original.value = to_jpeg_bytes(frame)
            image_processed.value = to_jpeg_bytes(processed)
            image_mask.value = to_jpeg_bytes(mask, is_mask=True)
        time.sleep(0.03)

# 播放/暂停按钮逻辑
def toggle_play(change):
    global playing
    playing = change['new']
    toggle_button.icon = 'pause' if playing else 'play'
    if playing:
        threading.Thread(target=play_video, daemon=True).start()

# 关闭按钮逻辑
def close_video(b):
    global playing
    playing = False
    toggle_button.value = False
    toggle_button.disabled = True
    close_button.disabled = True
    cap.release()
    # 清除画面
    image_original.value = b''
    image_processed.value = b''
    image_mask.value = b''

toggle_button.observe(toggle_play, names='value')
close_button.on_click(close_video)

# 滑块变化时更新当前帧
def refresh_on_change(change=None):
    with frame_lock:
        current = cap.get(cv2.CAP_PROP_POS_FRAMES)
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, current - 1))
        ret, frame = cap.read()
        if ret:
            processed, mask = process_frame(frame)
            image_original.value = to_jpeg_bytes(frame)
            image_processed.value = to_jpeg_bytes(processed)
            image_mask.value = to_jpeg_bytes(mask, is_mask=True)

for s in [slider_lowest, slider_red1_h, slider_red2_h]:
    s.observe(refresh_on_change, names='value')

# 布局
button_bar = widgets.HBox([toggle_button, close_button])
controls = widgets.VBox([button_bar, slider_lowest, slider_red1_h, slider_red2_h])
image_row = widgets.HBox([image_original, image_processed, image_mask])
display(controls, image_row)

# 初始化第一帧
refresh_on_change()


OSError: 无法打开视频文件